In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/master_degree/paper')
!ls

Mounted at /content/drive
 논문계획서_2nd.gdoc			  data
 archive.zip				 '기존 분석결과.gdoc'
 booking.com_hotel_review_korea.csv.zip  '논문 지도교수 배정신청서.gdoc'
 code_R					 '레퍼런스 링크.gdoc'
 colab


In [ ]:
import pandas as pd
from io import StringIO
import sys

In [ ]:
df = pd.read_csv('data/processed/rv_merge_en_1.csv', index_col=0)
df = df[~df.app_nm.isin(['naver', 'starbucks'])]

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
df.head()

,app_nm,review,rating,dt,app_ver,lang
0,zoom,ভাল,2.0,2020-12-28,5.2.45092.0831,en
1,zoom,This is our one of the education thing. We can...,5.0,2020-12-26,Unknown,en
2,zoom,Good,5.0,2020-12-26,5.4.6.812,en
3,zoom,Worst appp,2.0,2020-12-28,Unknown,en
4,zoom,Good,4.0,2020-12-28,5.4.7.946,en


# Dataset describe

In [ ]:
print(df['app_nm'].unique())
print('Number of apps: ',len(df['app_nm'].unique()))

['zoom' 'discord' 'fbm']
Number of apps:  3


In [ ]:
print('rows and columns', df.shape, '\n')
print('reviews of each app', df.groupby(['app_nm']).size())

rows and columns (1250664, 6) 

reviews of each app app_nm
discord    118292
fbm        795645
zoom       336727
dtype: int64
